## Про идентификацию имён

В первую очередь стоит отметить, что для корректности получаемых результатов было бы логичнее минимизировать ошибки второго рода, поскольку они могут быть исправлены вручную. 

В текущей реализации словаря ```natasha``` или аналоги не использованы, поскольку они могут порождать дополнительные ошибки, а также очень тяжеловесны в плане временных затрат. Единственный способ использования подобных инструментов видится в выделении отдельных элементов ФИО и их нормализация, но ввиду того что данные ожидаются нормализованными в этом нет большой необходиомсти.

Пока единственный вариант применения видится как проверка инициалов, где с помощью ```natasha``` извлекаются ИО и далее по словарю, который учитывает тип слова проверяется. Проверку желательно фамилий нужно будет проводить отдельн  

In [5]:
from utilities import names_dict
x = names_dict("resources\\names.xlsx")

print(x.get_names("Иннокентий"))
print(x.get_names("Хармс"))
print(x.get_names("Ахматова"))
print(x.get_names("Екатерина"))
print(x.get_names("Ахматова Екатерина Вадимовна"))

[]
['Борисович Александр Хармс']
['Ахматова Екатерина Вадимовна', 'Ахматова Александровна Екатерина']
['Пирогова Екатерина', 'Екатерина Грязнева', 'Ахматова Екатерина Вадимовна', 'Гумилева Екатерина Николаевна', 'Ахматова Александровна Екатерина', 'Щедрина Екатерина']
['Ахматова Екатерина Вадимовна']


In [6]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsNERTagger, PER, NamesExtractor, Doc)

def split_name(name : str) -> dict:
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    ner_tagger = NewsNERTagger(emb)
    morph_vocab = MorphVocab()
    names_extractor = NamesExtractor(MorphVocab())
    doc = Doc(name)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == PER:
            span.extract_fact(names_extractor)
    return {_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

print(split_name("Хармс АВ"))
print(split_name("Хармс А. В."))
print(split_name("Николай Андреевич Римский-Корсаков"))
print(split_name("Кузьма Сергеевич Петров-Водкин"))
print(split_name("Красавцев Иван Губинич"))
print(split_name("Орлов Иван Губинич"))
print(split_name("Ева-София Пономарева"))


{}
{'Хармс А. В.': {'first': 'А', 'last': 'Хармс', 'middle': 'В'}}
{'Николай Андреевич Римский-Корсаков': {'first': 'Николай', 'last': 'Римский', 'middle': 'Андреевич'}}
{'Кузьма Сергеевич Петров-Водкин': {'first': 'Кузьма', 'last': 'Петров', 'middle': 'Сергеевич'}}
{'Иван Губинич': {'first': 'Иван', 'last': 'Губинич'}}
{'Орлов Иван Губинич': {'first': 'Иван', 'last': 'Орлов'}}
{'Ева-София Пономарев': {'first': 'Ева'}}


In [3]:
from pymystem3 import Mystem

text = 'Николай Андреевич Римский-Корсаков'
def mystem_split_name(text : str) -> dict:
    m = Mystem()

    analyze = m.analyze(text)

    first_name = None
    second_name = None
    middle_name = None

    for word in analyze:
        try:
            analysis = word['analysis'][0]
        except KeyError:
            continue

        if 'имя' in analysis['gr']:
            first_name = word['text'].capitalize()
        elif 'фам' in analysis['gr']:
            second_name = word['text'].capitalize()
        elif 'отч' in analysis['gr']:
            middle_name = word['text'].capitalize()
    return {"first_name" : first_name, "middle_name" : middle_name, "second_name" : second_name}


print(mystem_split_name("Хармс АВ"))
print(mystem_split_name("Хармс А.В."))
print(mystem_split_name("Николай Андреевич Римский-Корсаков"))
print(mystem_split_name("Кузьма Сергеевич Петров-Водкин"))
print(mystem_split_name("Красавцев Иван Губинич"))
print(mystem_split_name("Орлов Иван Губинич"))
print(mystem_split_name("Ева-София Пономарева"))


{'first_name': 'Ав', 'middle_name': None, 'second_name': 'Хармс'}
{'first_name': None, 'middle_name': None, 'second_name': 'Хармс'}
{'first_name': 'Николай', 'middle_name': 'Андреевич', 'second_name': 'Корсаков'}
{'first_name': 'Кузьма', 'middle_name': 'Сергеевич', 'second_name': 'Водкин'}
{'first_name': 'Иван', 'middle_name': 'Губинич', 'second_name': None}
{'first_name': 'Иван', 'middle_name': 'Губинич', 'second_name': 'Орлов'}
{'first_name': 'Ева', 'middle_name': None, 'second_name': 'Пономарева'}


## Построение графиков

Собираем информацию из таблички

In [1]:
from preprocessing import Preprocessing
import nest_asyncio
nest_asyncio.apply()

data_path = "data.xlsx"
structure_path = "resources\\survey_structure.json"
names_path = "resources\\names.xlsx"

collector = Preprocessing(data_path, structure_path, names_path)
collector.collect()

collector.create_report_df()
#report.to_excel("output2.xlsx", index=False)
#print(report)
#collector.get_person_info("Илюхина Мария", "Непосредственный руководитель")


c:\Users\zalex\AI-analysis-of-free-survey-responses\preprocessing.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(data=[cur_row], columns = columns_names)], ignore_index = True)


,Имя,Помощь в решении проблемных вопросов,Эмоциональный настрой от совместной работы,Экспертность непосредственного руководителя,Вклад в улучшение и оптимизацию процессов подразделения,Способствование развитию команды,Процесс предоставления обратной связи,8. Поделитесь обратной связью для руководителя и предложениями по улучшению работы подразделения,Вклад в улучшение и оптимизацию процессов подразделения,Экспертность вышестоящего руководителя,Эмоциональный вклад,Предоставление информации по ключевым изменениям в компании,14. Поделитесь обратной связью для руководителя и предложениями по улучшению работы подразделения,17. Кем вам приходится выбранный руководитель?,Объединение кроссфункциональной команды,Эмоциональный настрой в команде,Экспертность функционального руководителя,Предоставление информации по ключевым изминениям в компании,22. Поделитесь обратной связью для руководителя проекта/продукта и предложениями по улучшению работы кроссфункциональной команды
0,Илюхина Мария,1.0,1.000000,9.333333,0.666667,0.666667,0.666667,"1. Повысить прозрачность рабочих процессов, об...",NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None
1,Ахматова Александровна Екатерина,1.0,1.000000,9.750000,1.000000,1.000000,1.000000,1. Необходимо время для полного выстраивания п...,1.0,10.0,1.0,1.0,1. Рекомендуется уделять больше внимания взаим...,None,NaN,NaN,NaN,NaN,None
2,Губин Павел,1.0,0.833333,7.666667,1.000000,0.833333,1.000000,1. Набрать персонал на вакантные должности в о...,NaN,NaN,NaN,NaN,None,None,1.0,1.0,10.0,1.0,None
3,Павел Шершнев,1.0,1.000000,10.000000,0.000000,1.000000,1.000000,1. Руководитель демонстрирует высокий уровень ...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None
4,Андрей Брэдбери,1.0,1.000000,9.333333,1.000000,1.000000,1.000000,"1. Рабочий процесс организован эффективно, пож...",NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Анастасия Толстова,1.0,1.000000,9.000000,1.000000,1.000000,1.000000,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None
111,Александр Гюго,1.0,1.000000,10.000000,1.000000,1.000000,1.000000,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None
112,Бронте Николай,NaN,NaN,NaN,NaN,NaN,NaN,None,1.0,10.0,1.0,1.0,None,None,NaN,NaN,NaN,NaN,None
113,Рейм Мария,1.0,1.000000,10.000000,1.000000,1.000000,1.000000,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None


In [3]:
collector.get_person_info("Илюхина Мария", "Непосредственный руководитель")

Помощь в решении проблемных вопросов_positive_pct                                                                                                 1.0
Помощь в решении проблемных вопросов_all                                                                                                            3
Эмоциональный настрой от совместной работы_positive_pct                                                                                           1.0
Эмоциональный настрой от совместной работы_all                                                                                                      3
Экспертность непосредственного руководителя_avg                                                                                              9.333333
Экспертность непосредственного руководителя_count                                                                                                   3
Вклад в улучшение и оптимизацию процессов подразделения_positive_pct                                

Определяем функцию, которая будет строить графики по для отдельных людей по выбранному типу руководства

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

def create_plot(name : str, group : str, collector : Preprocessing):
    person_stat = collector.get_select_vals_for_plot(name, group)
    colors = ['rgba(168,209,141,0.8)',
            'rgba(251,114,134,0.8)']

    x_data = person_stat[1]
    y_data = person_stat[0]
    fig = go.Figure()

    for i in range(0, len(x_data[0])):
        for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Bar(
                x=[xd[i]], y=[yd],
                orientation='h',
                marker=dict(
                    color=colors[i],
                    line=dict(width=0)
                )
            ))

    fig.update_layout(
        xaxis=dict(
            showgrid=True,
            showline=False,
            tickformat= '0.00%',
            showticklabels=True,
            color="white",
            zeroline=True,
            domain=[0.15, 1]
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
        ),
        barmode='stack',
        paper_bgcolor='rgb(0, 0, 0)',
        plot_bgcolor='rgb(0, 0, 0)',
        margin=dict(l=300, r=10, t=140, b=80),
        showlegend=False,
    )

    annotations = []

    for yd, xd in zip(y_data, x_data):
        # labeling the y-axis
        annotations.append(dict(xref='paper', yref='y',
                                x=0.14, y=yd,
                                xanchor='right',
                                text=str(yd),
                                font=dict(family='Arial', size=14,
                                        color='rgb(255, 255, 255)'),
                                showarrow=False, align='right'))
        # labeling the first percentage of each bar (x_axis)
        if xd[0]!=0:
            annotations.append(dict(xref='x', yref='y',
                                    x=xd[0] / 2, y=yd,
                                    text=str(int(xd[0]*100)) + '%',
                                    font=dict(family='Arial', size=14,
                                            color='rgb(0, 0, 0)'),
                                    showarrow=False))
        # labeling the second percentage of each bar (x_axis)
        if xd[1]!=0:
            annotations.append(dict(xref='x', yref='y',
                                    x=xd[0] + (xd[1]/2), y=yd,
                                    text=str(int(xd[1]*100)) + '%',
                                    font=dict(family='Arial', size=14,
                                                color='rgb(0, 0, 0)'),
                                    showarrow=False))

    fig.update_layout(annotations=annotations, title=dict(text=group, x=0.5 ,font=dict(size=50, color='rgb(255, 255, 255)')))
    fig.update_traces(width=0.5)
    fig.show()

По очереди строим примеры для непосредственных, вышестоящих и функциональных руководителей

In [5]:

create_plot("Губин Павел","Непосредственный руководитель", collector)

In [6]:
create_plot("Пушкин Иванович Валерий", "Вышестоящий руководитель", collector)

In [7]:
create_plot("Александр Альфонс", "Функциональный руководитель", collector)

Общая оценка по компании

In [8]:
print(collector.get_average_rating(4))

print(collector.get_average_rating([4,11]))

print(collector.get_average_rating([4,11,20]))

8.45374449339207
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Экспертность вышестоящего руководителя': 8.384615384615385, 'Overall': 8.433962264150944}
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Экспертность вышестоящего руководителя': 8.384615384615385, 'Экспертность функционального руководителя': 7.5, 'Overall': 8.338983050847459}
